# Advent 2024 d 10

The reindeer is holding a book titled "Lava Island Hiking Guide". However, when you open the book, you discover that most of it seems to have been scorched by lava! As you're about to ask how you can help, the reindeer brings you a blank topographic map of the surrounding area (your puzzle input) and looks up at you excitedly.

Perhaps you can help fill in the missing hiking trails?

The topographic map indicates the height at each position using a scale from 0 (lowest) to 9 (highest). For example:

```
0123
1234
8765
9876
```

Based on un-scorched scraps of the book, you determine that a good hiking trail is as long as possible and has an even, gradual, uphill slope. For all practical purposes, this means that a hiking trail is any path that starts at height 0, ends at height 9, and always increases by a height of exactly 1 at each step. Hiking trails never include diagonal steps - only up, down, left, or right (from the perspective of the map).

You look up from the map and notice that the reindeer has helpfully begun to construct a small pile of pencils, markers, rulers, compasses, stickers, and other equipment you might need to update the map with hiking trails.

A trailhead is any position that starts one or more hiking trails - here, these positions will always have height 0. Assembling more fragments of pages, you establish that a trailhead's score is the number of 9-height positions reachable from that trailhead via a hiking trail. In the above example, the single trailhead in the top left corner has a score of 1 because it can reach a single 9 (the one in the bottom left).

This trailhead has a score of 2:

```
...0...
...1...
...2...
6543456
7.....7
8.....8
9.....9
```

(The positions marked . are impassable tiles to simplify these examples; they do not appear on your actual topographic map.)

This trailhead has a score of 4 because every 9 is reachable via a hiking trail except the one immediately to the left of the trailhead:

```
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
```

This topographic map contains two trailheads; the trailhead at the top has a score of 1, while the trailhead at the bottom has a score of 2:

```
10..9..
2...8..
3...7..
4567654
...8..3
...9..2
.....01
```

Here's a larger example:

```
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
```

This larger example has 9 trailheads. Considering the trailheads in reading order, they have scores of 5, 6, 5, 3, 1, 3, 5, 3, and 5. Adding these scores together, the sum of the scores of all trailheads is 36.

The reindeer gleefully carries over a protractor and adds it to the pile. What is the sum of the scores of all trailheads on your topographic map?

In [ ]:
inp_sample = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

In [ ]:
from aocd import get_data
inp = get_data(day=10, year=2024)

from fastcore.all import *

## Reading notes and analysis:

- map, where every pos has a HEIGHT
- MOVEMENT is vertical or horizontal only
- TRAIL -- must be monotonically increasing in height, start at trailhead (0), and end at 9
- trailhead :: any position that starts one (or more!) trails (height == 0, and at an edge??)
  - trailhead score :: count of 9-height positions reachable via. a trail

Question: sum of scores of all trailheads?

### Analysis

- can find trails via BFS from trailhead.
- then exhaustively compute scores of all trailheads.

- optimizations: memoization to speed search over repeated path fragments??

## Plan:

Primitives needed:
- `neighbors_of_(node)` :: understands geometry
- `valid_neighbors_of_(node)` :: understnds geometry and monotonic height
- `paths_from_trailhead_(th:Node)`

In [ ]:
# Functions I plan to implement
from typing import Callable
Pos=tuple[int,int]
Map=dict[Pos,int]

def map_from_(mapstr:str) -> Map:
    ...

def neighbors_of_(pos:Pos,mmap:Map) -> list[Pos]:
    ...

def nondescending_neighbors_of_(pos:Pos,mmap:Map) -> list[Pos]:
    ...

def paths_from_trailhead_(th:Pos,mmap:Map) -> list[list[Pos]]:
    ...

def trailhead_score_(th:Pos,mmap:Map) -> int:
    ...

def bfs(start:Pos,neighbors_of:Callable[[Pos],list[Pos]]) -> list[Pos]:
    ...

In [ ]:
def map_from_(mapstr:str) -> Map:
    d = {(col,row):(int(h) if h.isdigit() else -1) for row,line in enumerate(mapstr.splitlines()) for col,h in enumerate(line)}
    return d

In [ ]:
map_from_(inp_sample)

{(0, 0): 8,
 (1, 0): 9,
 (2, 0): 0,
 (3, 0): 1,
 (4, 0): 0,
 (5, 0): 1,
 (6, 0): 2,
 (7, 0): 3,
 (0, 1): 7,
 (1, 1): 8,
 (2, 1): 1,
 (3, 1): 2,
 (4, 1): 1,
 (5, 1): 8,
 (6, 1): 7,
 (7, 1): 4,
 (0, 2): 8,
 (1, 2): 7,
 (2, 2): 4,
 (3, 2): 3,
 (4, 2): 0,
 (5, 2): 9,
 (6, 2): 6,
 (7, 2): 5,
 (0, 3): 9,
 (1, 3): 6,
 (2, 3): 5,
 (3, 3): 4,
 (4, 3): 9,
 (5, 3): 8,
 (6, 3): 7,
 (7, 3): 4,
 (0, 4): 4,
 (1, 4): 5,
 (2, 4): 6,
 (3, 4): 7,
 (4, 4): 8,
 (5, 4): 9,
 (6, 4): 0,
 (7, 4): 3,
 (0, 5): 3,
 (1, 5): 2,
 (2, 5): 0,
 (3, 5): 1,
 (4, 5): 9,
 (5, 5): 0,
 (6, 5): 1,
 (7, 5): 2,
 (0, 6): 0,
 (1, 6): 1,
 (2, 6): 3,
 (3, 6): 2,
 (4, 6): 9,
 (5, 6): 8,
 (6, 6): 0,
 (7, 6): 1,
 (0, 7): 1,
 (1, 7): 0,
 (2, 7): 4,
 (3, 7): 5,
 (4, 7): 6,
 (5, 7): 7,
 (6, 7): 3,
 (7, 7): 2}

In [ ]:
inp_sample

'89010123\n78121874\n87430965\n96549874\n45678903\n32019012\n01329801\n10456732'

In [ ]:
def maxes(mmap:Map) -> Tuple[int,int]:
    maxX = max(x for x,_ in mmap.keys())
    maxY = max(y for _,y in mmap.keys())
    return maxX,maxY

def on_map_(mmap:Map,n:Pos):
    maxX,maxY = maxes(mmap)
    return 0 <= n[0] <= maxX and 0 <= n[1] <= maxY

def neighbors_of_(mmap:Map, pos:Pos) -> List[Pos]:
    maxX,maxY = maxes(mmap)
    neighbors = ((pos[0]+dx,pos[1]+dy) for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)])
    neighbors = (n for n in neighbors if on_map_(mmap,n))
    return list(neighbors)

def nondescending_neighbors_of_(mmap:Map,pos:Pos) -> List[Pos]:
    return [n for n in neighbors_of_(mmap,pos) if mmap[n] >= mmap[pos]]

In [ ]:
nondescending_neighbors_of_(map_from_(inp_sample),(1,3))

[(0, 3), (1, 2)]

In [ ]:
the_map = map_from_(inp_sample)
the_map

{(0, 0): 8,
 (1, 0): 9,
 (2, 0): 0,
 (3, 0): 1,
 (4, 0): 0,
 (5, 0): 1,
 (6, 0): 2,
 (7, 0): 3,
 (0, 1): 7,
 (1, 1): 8,
 (2, 1): 1,
 (3, 1): 2,
 (4, 1): 1,
 (5, 1): 8,
 (6, 1): 7,
 (7, 1): 4,
 (0, 2): 8,
 (1, 2): 7,
 (2, 2): 4,
 (3, 2): 3,
 (4, 2): 0,
 (5, 2): 9,
 (6, 2): 6,
 (7, 2): 5,
 (0, 3): 9,
 (1, 3): 6,
 (2, 3): 5,
 (3, 3): 4,
 (4, 3): 9,
 (5, 3): 8,
 (6, 3): 7,
 (7, 3): 4,
 (0, 4): 4,
 (1, 4): 5,
 (2, 4): 6,
 (3, 4): 7,
 (4, 4): 8,
 (5, 4): 9,
 (6, 4): 0,
 (7, 4): 3,
 (0, 5): 3,
 (1, 5): 2,
 (2, 5): 0,
 (3, 5): 1,
 (4, 5): 9,
 (5, 5): 0,
 (6, 5): 1,
 (7, 5): 2,
 (0, 6): 0,
 (1, 6): 1,
 (2, 6): 3,
 (3, 6): 2,
 (4, 6): 9,
 (5, 6): 8,
 (6, 6): 0,
 (7, 6): 1,
 (0, 7): 1,
 (1, 7): 0,
 (2, 7): 4,
 (3, 7): 5,
 (4, 7): 6,
 (5, 7): 7,
 (6, 7): 3,
 (7, 7): 2}

In [ ]:
maxes(the_map)

(7, 7)

In [ ]:
def bfs(mmap:Map,start:Pos) -> dict[Pos,list]:
    seen:dict[Pos,list] = dict()
    tosee = L([(start,[])]) # (Pos,path)
    while tosee:
        parent,parent_path = tosee.pop(0)
        seen[parent]=parent_path
        children = [(child,(parent_path + [i]))
                        for i,child in enumerate(nondescending_neighbors_of_(mmap,parent)) 
                        if child not in seen and child not in tosee.map(lambda pr:pr[0])]
        for child in children:
            tosee.append(child)
    return seen

In [ ]:
basic_map_src = """0123
1234
8765
9876"""

In [ ]:
bfs(map_from_(basic_map_src),(0,0))

{(0, 0): [],
 (1, 0): [0],
 (0, 1): [1],
 (2, 0): [0, 0],
 (1, 1): [0, 1],
 (0, 2): [1, 1],
 (3, 0): [0, 0, 0],
 (2, 1): [0, 0, 1],
 (1, 2): [0, 1, 1],
 (0, 3): [1, 1, 0],
 (3, 1): [0, 0, 0, 0],
 (2, 2): [0, 0, 1, 1],
 (1, 3): [0, 1, 1, 1],
 (3, 2): [0, 0, 0, 0, 0],
 (2, 3): [0, 0, 1, 1, 1],
 (3, 3): [0, 0, 0, 0, 0, 1]}

**AI Prompt**

Can you explain the error above? Is it due to my version of Python or would I get it in any version?

**AI Response**

The error is happening because of a syntax issue in the `bfs` function that would occur in any Python version. The problem is in this line:

```python
seen = dict[Pos,list]
```

This line is trying to use type annotation syntax (`dict[Pos,list]`), but it's actually creating a type hint object rather than an empty dictionary. To fix this, you should instead write:

```python
seen = {}  # or seen = dict()
```

If you want to keep the type hint for better code documentation, you could write it as:

```python
seen: dict[Pos,list] = {}
```

Would you like me to explain more about Python type hints, or would you prefer to try fixing this error first?

In [ ]:
def get_height_(mmap:Map,pos:Pos): return mmap[pos]

### re-read!

Just realized I misunderstood something. trails height must increase with every step, not just not decrease.

This exclude paths with steps were the height stays the same.

It also removes the question of how to count paths which have multiple steps all at height 9 at the end.

In [ ]:
def stepup_neighbors_of_(mmap:Map,pos:Pos) -> List[Pos]:
    return [n for n in neighbors_of_(mmap,pos) if mmap[n] == 1+ mmap[pos]]

In [ ]:
def bfs(mmap:Map,start:Pos) -> dict[Pos,list]:
    seen:dict[Pos,list] = dict()
    tosee = L([(start,[])]) # (Pos,path)
    while tosee:
        parent,parent_path = tosee.pop(0)
        seen[parent]=parent_path
        children = [(child,(parent_path + [i]))
                        for i,child in enumerate(stepup_neighbors_of_(mmap,parent)) 
                        if child not in seen and child not in tosee.map(lambda pr:pr[0])]
        for child in children:
            tosee.append(child)
    return seen

In [ ]:
def trails_from_trailhead(mmap:Map,th:Pos):
    paths = bfs(mmap,th)
    trailscores = []
    for end,route in paths.items():
        if get_height_(mmap,end) == 9:
            trailscores.append((end,route,len(route)))
    return trailscores

In [ ]:
def score_trailhead(mmap:Map,th:Pos):
    return len(trails_from_trailhead(mmap,th))

In [ ]:
trails_from_trailhead(map_from_(basic_map_src),(0,0))

[((0, 3), [0, 0, 0, 0, 0, 0, 0, 0, 0], 9)]

In [ ]:
def trailheads(mmap:Map):
    return [p for p in mmap.keys() if get_height_(mmap,p) == 0]

In [ ]:
trailheads(map_from_(inp_sample))

[(2, 0), (4, 0), (4, 2), (6, 4), (2, 5), (5, 5), (0, 6), (6, 6), (1, 7)]

In [ ]:
sample_map = map_from_(inp_sample)
[score_trailhead(sample_map,th) for th in trailheads(sample_map)]

[5, 6, 5, 3, 1, 3, 5, 3, 5]

In [ ]:
inp_map = map_from_(inp)
sum(score_trailhead(inp_map,th) for th in trailheads(inp_map))

796

In [ ]:
len(trailheads(inp_map))

0

In [ ]:
len(inp_map)

22886

In [ ]:
L(trailheads(inp_map))

(#0) []

In [ ]:
L(inp_map.keys())

(#22886) [(0, 0),(1, 0),(2, 0),(3, 0),(4, 0),(5, 0),(6, 0),(7, 0),(8, 0),(9, 0),(10, 0),(11, 0),(12, 0),(13, 0),(14, 0),(15, 0),(16, 0),(17, 0),(18, 0),(19, 0)...]

In [ ]:
get_height_(inp_map,(0,0))

-1

In [ ]:
len(trailheads(inp_map))

0

**AI Prompt**

Can you see what is wrong with my code above? I am not seeing any trailheads in the inp_map

**AI Response**

Let's check what's happening with your input data. I notice that `get_height_(inp_map,(0,0))` returns `-1`, which suggests there might be an issue with how the input is being read. 

In your `map_from_` function, you have this line:
```python
d = {(col,row):(int(h) if h.isdigit() else -1) for row,line in enumerate(mapstr.splitlines()) for col,h in enumerate(line)}
```

Could you print out the first few lines of your `inp` variable so we can see what the raw input looks like? You can do this with:
```python
print(inp[:100])
```

This will help us determine if there's an issue with the input format or if we need to adjust the parsing function.

In [ ]:
print(inp[:10])

wubu, brr,


Good suggestion. I was importing input from the wrong problem

## Part 2

- score:  number of 9-heights positions which were reachable.
- rating number of distinct trails beginning at the trailhead.
- find: sum of all ratings

Okay. so

old problem was to count reachable 9s.
new problem is to count paths to 9s. 

old search was BFS, tracking the path:
- marking items seen prevented backtracking but also 
- did it also prevent counting paths? I don't think so. let's experiment.

In [ ]:
maps1="""
.....0.
..4321.
..5..2.
..6543.
..7..4.
..8765.
..9....
""".strip()
map1 = map_from_(maps1)

In [ ]:
trailheads(map1)

[(5, 0)]

In [ ]:
paths_from_trailhead_(map1, (5, 0))

In [ ]:
trails_from_trailhead(map1, (5, 0))

[((2, 6), [0, 0, 0, 0, 0, 0, 0, 0, 0], 9)]

In [ ]:
bfs(map1, (5, 0))

{(5, 0): [],
 (5, 1): [0],
 (4, 1): [0, 0],
 (5, 2): [0, 1],
 (3, 1): [0, 0, 0],
 (5, 3): [0, 1, 0],
 (2, 1): [0, 0, 0, 0],
 (4, 3): [0, 1, 0, 0],
 (5, 4): [0, 1, 0, 1],
 (2, 2): [0, 0, 0, 0, 0],
 (3, 3): [0, 1, 0, 0, 0],
 (5, 5): [0, 1, 0, 1, 0],
 (2, 3): [0, 0, 0, 0, 0, 0],
 (4, 5): [0, 1, 0, 1, 0, 0],
 (2, 4): [0, 0, 0, 0, 0, 0, 0],
 (3, 5): [0, 1, 0, 1, 0, 0, 0],
 (2, 5): [0, 0, 0, 0, 0, 0, 0, 0],
 (2, 6): [0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
maps1="""
.....0.
..4321.
..5..2.
..6543.
..7..4.
..8765.
..9....
""".strip()
map1 = map_from_(maps1)

BFS does not capture multiple paths since:
- once 9 is visited once, it will not be added again.
- if two path prefixes lead to the same intermediate point, it will only be added once. 

But, the requirements that steps be ascending suffices to prevent loops

In [ ]:
def path_searchs(mmap:Map,start:Pos):
    seen:list[(Pos,list)] = []
    tosee = L([(start,[])]) # (Pos,path)
    while tosee:
        parent,parent_path = tosee.pop(0)
        seen.append((parent,parent_path))
        childpaths = [(child,(parent_path + [i]))
                        for i,child in enumerate(stepup_neighbors_of_(mmap,parent))]
        for childpath in childpaths:
            tosee.append(childpath)
    return seen

In [ ]:
path_searchs(map1, (5, 0))

[((5, 0), []),
 ((5, 1), [0]),
 ((4, 1), [0, 0]),
 ((5, 2), [0, 1]),
 ((3, 1), [0, 0, 0]),
 ((5, 3), [0, 1, 0]),
 ((2, 1), [0, 0, 0, 0]),
 ((4, 3), [0, 1, 0, 0]),
 ((5, 4), [0, 1, 0, 1]),
 ((2, 2), [0, 0, 0, 0, 0]),
 ((3, 3), [0, 1, 0, 0, 0]),
 ((5, 5), [0, 1, 0, 1, 0]),
 ((2, 3), [0, 0, 0, 0, 0, 0]),
 ((2, 3), [0, 1, 0, 0, 0, 0]),
 ((4, 5), [0, 1, 0, 1, 0, 0]),
 ((2, 4), [0, 0, 0, 0, 0, 0, 0]),
 ((2, 4), [0, 1, 0, 0, 0, 0, 0]),
 ((3, 5), [0, 1, 0, 1, 0, 0, 0]),
 ((2, 5), [0, 0, 0, 0, 0, 0, 0, 0]),
 ((2, 5), [0, 1, 0, 0, 0, 0, 0, 0]),
 ((2, 5), [0, 1, 0, 1, 0, 0, 0, 0]),
 ((2, 6), [0, 0, 0, 0, 0, 0, 0, 0, 0]),
 ((2, 6), [0, 1, 0, 0, 0, 0, 0, 0, 0]),
 ((2, 6), [0, 1, 0, 1, 0, 0, 0, 0, 0])]

In [ ]:
def distinct_trails_from_trailhead(mmap:Map,th:Pos):
    dts = []
    for end,path in path_searchs(mmap,th):
        if get_height_(mmap,end) == 9:
            dts.append((end,path))
    return dts

In [ ]:
distinct_trails_from_trailhead(map1, (5, 0))

[((2, 6), [0, 0, 0, 0, 0, 0, 0, 0, 0]),
 ((2, 6), [0, 1, 0, 0, 0, 0, 0, 0, 0]),
 ((2, 6), [0, 1, 0, 1, 0, 0, 0, 0, 0])]

In [ ]:
def rating_trailhead(mmap:Map,th:Pos):
    return len(distinct_trails_from_trailhead(mmap,th))

In [ ]:
def rate_map(mmap:Map):
    return sum(rating_trailhead(mmap,th) for th in trailheads(mmap))

In [ ]:
inp_sample

'89010123\n78121874\n87430965\n96549874\n45678903\n32019012\n01329801\n10456732'

In [ ]:
rate_map( sample_map )

81

In [ ]:
rate_map( inp_map )

1942

> **meta-analysis**: 
> 1. Yet another problem where I would have gone faster by reading the description more slowly at first!
>
> 2. writing function signatures is a decent way to outline a plan, but also confuses autcomplete systems.
> 3. Using functional abstraction might be a good way to support looping back to add memoization or to optimize DSs later.
> 
> More generally, I'm struck by the extent to which much of the friction is due to stupid mistakes. It seems like a good mental model would be to imagine that there is something wrong with your keyboard, so that everything which you type has a 20% of having an error, not through your fault, but just due to the medium. In this case, it does not matter how clearly you apprehend in your mind what you intend to type. You would still need to proceed very incrementally, because of the gap between your intentions and what makes it into the computer.